In [1]:
import pandas as pd
import geopandas as gpd
import os
import numpy as np
from operator import add

from shapely import wkt
from shapely.geometry import shape, LineString, Point
    
# funciones desarrolladas
from functions.agrupar_dfs_censo import *
from functions.cargar_data import *
from functions.impresion import *

In [2]:
# carga datos diádicos
pbi = cargar_pbi()
md = cargar_matriz_distancias()
vecindad = cargar_vecindad()

# carga capas de fatos geográficos
deptos, deptos_sim, localidad, centro_pobl, capital = cargar_datos_geo()

# carga migrantes internos
p_migr = cargar_migrantes_internos()

# carga poblacion por depto
pobl = recuperar_poblacion_2011()

/home/guillermo/anaconda3/envs/tesis/lib/python3.9/site-packages/geopandas/geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


In [3]:
# genera un sólo dataframe solo para deptos
flujos_deptos = p_migr[['depto_origen', 'depto_destino']]

flujos_deptos = flujos_deptos[flujos_deptos.depto_origen != flujos_deptos.depto_destino]

flujos_deptos['personas_mig'] = 1

# agrupa y cuenta
grupo = flujos_deptos.groupby(by=['depto_origen', 'depto_destino']).sum()

print('Cantidad de díadas ', grupo.shape[0])

grupo.head(3)

Cantidad de díadas  342


personas_mig
depto_origen depto_destino              
1            2                       914
             3                     33127
             4                      1387

In [4]:
# genera tabla pivot con los flujos de departamento a departamento
matrix = pd.pivot_table(flujos_deptos,
                        index ='depto_origen',
                        columns='depto_destino',
                        fill_value=0,
                        aggfunc=sum,
                        margins=True,
                        margins_name='Total')

In [5]:
# # guarda en csv
# matrix = pd.pivot_table(flujos_deptos,
#                         index ='depto_origen',
#                         columns='depto_destino',
#                         fill_value=0,
#                         aggfunc=sum,
#                         margins=False)

# matrix.to_csv('tablas/matriz_deptos.csv', header=True)

# # guarda en latex
# # lista de nombres deptos para generar tabla en latex
# nomdep = ['Mvdeo.', 'Artigas', 'Can.', 'C. Largo', 'Colonia', 'Durazno',
#           'Flores', 'Florida', 'Lavalleja','Maldonado', 'Paysandú', 'R. Negro', 'Rivera',
#           'Rocha', 'Salto', 'San José', 'Soriano', 'Tacuarembó', 'T. y Tres']

# coddep = np.arange(1, 20, 1).tolist()

# # guarda tabla en latex
# # cabecera = datos_dpto.sort_values('DPTO')['NOMBRE'].to_list()
# # cabecera.append('Total')

# matrix_tex = matrix
# matrix_tex.rename(index=dict(zip(coddep, nomdep)), inplace=True)

# # setea ancho de columnas
# ancho = 'p{0.7cm}'
# colformato='l' + ancho * 20

# matrix_tex.to_latex(buf= "tablas/matriz_orig_dest.tex", bold_rows=False,
#                     column_format = colformato,
#                     caption= 'Matriz de movimientos entre departamentos (Censo INE 2011).')

In [6]:
# unimos todo en un dataframe de díadas

# migrantes
df_agrupado = grupo.reset_index()

# agrega codigo único
df_agrupado.insert(0, 'cod', (df_agrupado['depto_origen'].astype(str)
                   + df_agrupado['depto_destino'].astype(str).str.zfill(2)).astype(int))

df_agrupado.head()

,cod,depto_origen,depto_destino,personas_mig
0,102,1,2,914
1,103,1,3,33127
2,104,1,4,1387
3,105,1,5,2100
4,106,1,6,982


In [7]:
# pega masas (población!)
df_agrupado = df_agrupado.merge(pobl, how='left', left_on='depto_origen', right_on='DPTO').drop('DPTO', axis=1).rename(columns={'poblacion':'pob_origen'})

df_agrupado = df_agrupado.merge(pobl, how='left', left_on='depto_destino', right_on='DPTO').drop('DPTO', axis=1).rename(columns={'poblacion':'pob_destino'})

df_agrupado.head()

,cod,depto_origen,depto_destino,personas_mig,pob_origen,pob_destino
0,102,1,2,914,1318755,73377
1,103,1,3,33127,1318755,520173
2,104,1,4,1387,1318755,84698
3,105,1,5,2100,1318755,123203
4,106,1,6,982,1318755,57084


In [8]:
# pega matriz de distancias
df_agrupado = df_agrupado.merge(md, how='left', on='cod').drop(['cod_ori', 'cod_des'], axis=1).rename(columns={'distancia':'dist'})
df_agrupado.head()

,cod,depto_origen,depto_destino,personas_mig,pob_origen,pob_destino,dist
0,102,1,2,914,1318755,73377,610726
1,103,1,3,33127,1318755,520173,22544
2,104,1,4,1387,1318755,84698,393703
3,105,1,5,2100,1318755,123203,172773
4,106,1,6,982,1318755,57084,198970


In [9]:
# pega datos del PBI
df_agrupado = df_agrupado.merge(pbi, how='left', left_on='depto_origen', right_on='DPTO').drop(['DPTO'], axis=1).rename(columns={'miles_de_pesos':'pbi_origen', 'porcentaje_pbi': 'pbi_porcen_ori'})

df_agrupado = df_agrupado.merge(pbi, how='left', left_on='depto_destino', right_on='DPTO').drop(['DPTO'], axis=1).rename(columns={'miles_de_pesos':'pbi_destino', 'porcentaje_pbi': 'pbi_porcen_des'})

df_agrupado.head()

,cod,depto_origen,depto_destino,personas_mig,pob_origen,pob_destino,dist,pbi_origen,pbi_porcen_ori,pbi_destino,pbi_porcen_des
0,102,1,2,914,1318755,73377,610726,465848031,"50,3",14214980,"1,5"
1,103,1,3,33127,1318755,520173,22544,465848031,"50,3",86306492,"9,3"
2,104,1,4,1387,1318755,84698,393703,465848031,"50,3",17048887,"1,8"
3,105,1,5,2100,1318755,123203,172773,465848031,"50,3",43788749,"4,7"
4,106,1,6,982,1318755,57084,198970,465848031,"50,3",13023942,"1,4"


In [10]:
# pega vecindad
df_agrupado = df_agrupado.merge(vecindad, how='left', on='cod').drop(['dep1', 'dep2'], axis=1)
df_agrupado['dummy_limit'] = True
df_agrupado.loc[df_agrupado.largo_limite.isnull(), 'dummy_limit'] = False

dd_deptos = df_agrupado

print(df_agrupado.shape)
df_agrupado.head()

(342, 13)


,cod,depto_origen,depto_destino,personas_mig,pob_origen,pob_destino,dist,pbi_origen,pbi_porcen_ori,pbi_destino,pbi_porcen_des,largo_limite,dummy_limit
0,102,1,2,914,1318755,73377,610726,465848031,"50,3",14214980,"1,5",NaN,False
1,103,1,3,33127,1318755,520173,22544,465848031,"50,3",86306492,"9,3",60107.0,True
2,104,1,4,1387,1318755,84698,393703,465848031,"50,3",17048887,"1,8",NaN,False
3,105,1,5,2100,1318755,123203,172773,465848031,"50,3",43788749,"4,7",NaN,False
4,106,1,6,982,1318755,57084,198970,465848031,"50,3",13023942,"1,4",NaN,False


In [11]:
# calcula logaritmos
def logaritmos(df, variables):
    new_vars = ['log_' + sub for sub in variables]
    nvars = len(variables)
    for i in range(nvars):
        df[new_vars[i]] = np.log(df[variables[i]])
    return df

# definir lista de variables sobre las cuales se quiere el logaritmo
variables = ['pbi_destino', 'dist']

dd_deptos = logaritmos(dd_deptos, variables)

dd_deptos.head()

,cod,depto_origen,depto_destino,personas_mig,pob_origen,pob_destino,dist,pbi_origen,pbi_porcen_ori,pbi_destino,pbi_porcen_des,largo_limite,dummy_limit,log_pbi_destino,log_dist
0,102,1,2,914,1318755,73377,610726,465848031,"50,3",14214980,"1,5",NaN,False,16.469807,13.322404
1,103,1,3,33127,1318755,520173,22544,465848031,"50,3",86306492,"9,3",60107.0,True,18.273415,10.023224
2,104,1,4,1387,1318755,84698,393703,465848031,"50,3",17048887,"1,8",NaN,False,16.651595,12.883352
3,105,1,5,2100,1318755,123203,172773,465848031,"50,3",43788749,"4,7",NaN,False,17.594887,12.059734
4,106,1,6,982,1318755,57084,198970,465848031,"50,3",13023942,"1,4",NaN,False,16.382300,12.200909


In [12]:
# decodifica el departamento (códigos INE)
dd_deptos['nom_depto_orig'] = decode_depto(dd_deptos, 'depto_origen')
dd_deptos['nom_depto_des'] = decode_depto(dd_deptos, 'depto_destino')

dd_deptos.head()

,cod,depto_origen,depto_destino,personas_mig,pob_origen,pob_destino,dist,pbi_origen,pbi_porcen_ori,pbi_destino,pbi_porcen_des,largo_limite,dummy_limit,log_pbi_destino,log_dist,nom_depto_orig,nom_depto_des
0,102,1,2,914,1318755,73377,610726,465848031,"50,3",14214980,"1,5",NaN,False,16.469807,13.322404,MONTEVIDEO,ARTIGAS
1,103,1,3,33127,1318755,520173,22544,465848031,"50,3",86306492,"9,3",60107.0,True,18.273415,10.023224,MONTEVIDEO,CANELONES
2,104,1,4,1387,1318755,84698,393703,465848031,"50,3",17048887,"1,8",NaN,False,16.651595,12.883352,MONTEVIDEO,CERRO LARGO
3,105,1,5,2100,1318755,123203,172773,465848031,"50,3",43788749,"4,7",NaN,False,17.594887,12.059734,MONTEVIDEO,COLONIA
4,106,1,6,982,1318755,57084,198970,465848031,"50,3",13023942,"1,4",NaN,False,16.382300,12.200909,MONTEVIDEO,DURAZNO


In [13]:
# guarda
dd_deptos.to_csv('tablas/dd_deptos.csv', index=False)